### Install ngrok library

In [ ]:
!pip install fastapi
!pip install uvicorn
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio
!pip install appwrite



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 15.3 MB/s eta 0:00:00
  Cloning https://github.com/facebookresearch/audiocraft.git to /tmp/pip-req-build-q2nwu5kl
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/audiocraft.git /tmp/pip-req-build-q2nwu5kl
  Resolved https://github.com/facebookresearch/audiocraft.git to commit adf0b04a4452f171970028fcf80f101dd5e26e19
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 6.8 MB/s

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import torch

from fastapi.middleware.cors import CORSMiddleware
from io import BytesIO
import base64
from PIL import Image

app = FastAPI()

# CORS configuration
origins = [
    "http://localhost:3000",  # Add the URL of your frontend app
    "http://127.0.0.1:3000",  # Localhost may also use this URL
    # "https://yourproductiondomain.com",  # Uncomment this line for your production domain if necessary
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,  # List of allowed origins
    allow_credentials=True,
    allow_methods=["*"],  # Allows all methods
    allow_headers=["*"],  # Allows all headers
)

# Text-to-image api endpoint

# hugging face login

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install diffusers

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
# from type import List


class stable_diffusion(BaseModel):
    prompt: str
    height: int
    width: int
    num_inference_steps: int
    guidance_scale: float
    num_images_per_prompt: int
    # negative_prompt:str
    # timesteps: List[int]
    # sigmas: List[float]

model_id = "stabilityai/stable-diffusion-2-1"

# Use the DPMSolverMultistepScheduler (DPM-Solver++) scheduler here instead
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

def text_to_image(input):
    image = pipe(
        input.prompt,
        height = input.height,
        width = input.width,
        negative_prompt="",
        num_inference_steps=input.num_inference_steps,
        guidance_scale= input.guidance_scale,
        num_images_per_prompt=input.num_images_per_prompt,
        # timesteps = input.timesteps,
        # sigmas = input.sigmas
    ).images[0]

    # Convert the image to a base64 string
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")

    return img_str

@app.post("/api/text_to_image")
async def api_text_to_image(item: stable_diffusion):
    result = text_to_image(item)
    return {"output": result}


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

# Chattts API end point

In [ ]:

!pip install ChatTTS
!pip install numpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.8/114.8 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 k

In [ ]:
import torch
import ChatTTS
import torchaudio
from IPython.display import Audio
import numpy as np

class chatTTS(BaseModel):
    input: str
    temperature: float ## 0.00001 - 1
    top_P: float ## 0.1-0.9
    top_K: int ## 1- 20
    oral: int ##
    laugh: int ##
    break_: int ##
    seed: int

chat = ChatTTS.Chat()
chat.load(compile=False) # Set to True for better performance

def deterministic(seed=0):
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def generate_audio_from_text(user_input):
    torch.manual_seed(user_input.seed)
    # deterministic(user_input.seed)
    rand_spk = chat.sample_random_speaker()
    params_infer_code = ChatTTS.Chat.InferCodeParams(
      spk_emb = rand_spk, # add sampled speaker
      temperature = user_input.temperature,   # using custom temperature
      top_P = user_input.top_P,        # top P decode
      top_K = user_input.top_K,         # top K decode
    )

    ###################################
    # For sentence level manual control.

    # use oral_(0-9), laugh_(0-2), break_(0-7)
    # to generate special token in text to synthesize.
    params_refine_text = ChatTTS.Chat.RefineTextParams(
        prompt=f'[oral_{user_input.oral}][laugh_{user_input.laugh}][break_{user_input.break_}]',
    )

    wavs = chat.infer(
        user_input.input,
        skip_refine_text=True,
        params_refine_text=params_refine_text,
        params_infer_code=params_infer_code,
    )

    audio_file = BytesIO()
    torchaudio.save(audio_file, torch.tensor(wavs[0]), 24000, format="wav")
    audio_file.seek(0)
    audio_base64 = base64.b64encode(audio_file.read()).decode('utf-8')

    return audio_base64

@app.post("/api/generate_audio")
async def api_generate_audio(item: chatTTS):
    audio = generate_audio_from_text(item)
    return {"output": audio}


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 577, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/lib/python3.10/asyncio/tasks.py", l

# **Animated Diff API endpoint** Working on this rn

---



In [ ]:
import torch
from diffusers import AnimateDiffPipeline, MotionAdapter, EulerDiscreteScheduler
from diffusers.utils import export_to_gif
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file

device = "cuda"
dtype = torch.float16

step = 4  # Options: [1,2,4,8]
repo = "ByteDance/AnimateDiff-Lightning"
ckpt = f"animatediff_lightning_{step}step_diffusers.safetensors"
base = "emilianJR/epiCRealism"  # Choose to your favorite base model.

adapter = MotionAdapter().to(device, dtype)
adapter.load_state_dict(load_file(hf_hub_download(repo ,ckpt), device=device))
pipe_ = AnimateDiffPipeline.from_pretrained(base, motion_adapter=adapter, torch_dtype=dtype).to(device)
pipe_.scheduler = EulerDiscreteScheduler.from_config(pipe_.scheduler.config, timestep_spacing="trailing", beta_schedule="linear")


ModuleNotFoundError: No module named 'diffusers'

In [ ]:
class animate(BaseModel):
    input: str

def generate_animated_gif(user_prompt):
    # Assuming the output from the pipeline is already fetched
    output = pipe_(prompt=user_prompt, guidance_scale=1.0, num_inference_steps=step)

    # If output.frames is a list of lists of PIL Images
    if not output.frames or not output.frames[0]:
        raise ValueError("No frames generated")

    # Flatten the list if it's nested and filter out any non-PIL Image items
    frames = [img for sublist in output.frames for img in sublist if isinstance(img, Image.Image)]

    if not frames:
        raise ValueError("No valid frames were found in the output")

    # Save frames as GIF
    gif_file = BytesIO()
    frames[0].save(gif_file, format='GIF', save_all=True, append_images=frames[1:], loop=0, duration=100)
    gif_file.seek(0)
    gif_base64 = base64.b64encode(gif_file.read()).decode('utf-8')

    return gif_base64

@app.post("/api/generate_gif")
async def api_generate_audio(item: animate):
    gif = generate_animated_gif(item.input)
    return {"output": gif}

# **Music**

In [ ]:
from transformers import AutoProcessor, MusicgenForConditionalGeneration
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write


class music(BaseModel):
  input: str

processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")

@app.post("/api/generate_music")
async def generate_music(item: music):
        inputs = processor(
            text=[item.input],
            padding=True,
            return_tensors="pt",
        )

        # Generate audio values
        audio_values = model.generate(**inputs, max_new_tokens=256)

        audio_file = BytesIO()
        torchaudio.save(audio_file, torch.tensor(audio_values[0]), 24000, format="wav")
        audio_file.seek(0)
        audio_base64 = base64.b64encode(audio_file.read()).decode('utf-8')

        return {"output": audio_base64}

In [ ]:
!ngrok config add-authtoken 2jRZ4PPjASkmxNIA5HVzbbpOIZt_3Db1d65vWC1JRZzYzN9ta

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


# Image to Video

In [ ]:
import torch

from diffusers import StableVideoDiffusionPipeline
from diffusers.utils import load_image, export_to_video

video_pipe = StableVideoDiffusionPipeline.from_pretrained(
    "stabilityai/stable-video-diffusion-img2vid-xt", torch_dtype=torch.float16, variant="fp16"
)
video_pipe.enable_model_cpu_offload()

# image = load_image("https://cloud.appwrite.io/v1/storage/buckets/66ab98e800074cb72188/files/8wwxaa985fgdaf35761n/view?project=66ab7c0f0031bd4ae2ac&mode=admin")
# image = image.resize((1024, 576))

# generator = torch.manual_seed(42)
# frames =video_pipe(image, decode_chunk_size=8, generator=generator).frames[0]

# export_to_video(frames, "generated.mp4", fps=7)


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
from appwrite.client import Client
from appwrite.input_file import InputFile
from appwrite.services.storage import Storage
import random
import string

In [ ]:
class video(BaseModel):
    imageUrl:str

project_id = "66ab7c0f0031bd4ae2ac"
video_bucket_id = "66ae1dd8000f785812d9"
client = Client()
client.set_endpoint('https://cloud.appwrite.io/v1')  # Your API Endpoint
client.set_project(project_id)  # Your project ID
client.set_session('')  # The user session to authenticate with
storage = Storage(client)

def generate_unique_id_v2(length=20):
    characters = string.ascii_lowercase + string.digits
    unique_id = ''.join(random.choice(characters) for _ in range(length))
    return unique_id

def getFileUrl(file_id,bucket_id):
    base_url = "https://cloud.appwrite.io/v1/storage/buckets/"
    url = f"{base_url}{bucket_id}/files/{file_id}/view?project={project_id}&mode=admin"
    return url

def generateVideoFileUrl(filePath):
    fileId=generate_unique_id_v2()
    try:
        result = storage.create_file(
            bucket_id=video_bucket_id,
            file_id=fileId,
            file=InputFile.from_path(filePath),

        )
        url = getFileUrl(fileId,video_bucket_id)
        return url
    except:
        return None



def generate_video(imageUrl):
    image = load_image(imageUrl)
    image = image.resize((1024, 576))
    generator = torch.manual_seed(42)
    frames = video_pipe(image, decode_chunk_size=8, generator=generator,motion_bucket_id=180, noise_aug_strength=0.1).frames[0]
    # export_to_video(frames, "generated.mp4", fps=7)


    if not frames:
        raise ValueError("No valid frames were found in the output")

    # Save frames as GIF
    video_path = export_to_video(frames, "generated.mp4", fps=7)


    url=generateVideoFileUrl(video_path)

    return url

@app.post("/api/generate_video")
async def api_generate_video(item: video):
    video_url = generate_video(item.imageUrl)
    return {"output": video_url}

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-72' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 577, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/lib/python3.10/asyncio/tasks.py", 

In [ ]:
!pip install -q -U diffusers transformers accelerate

In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)

nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://dd71-34-126-157-248.ngrok-free.app


INFO:     Started server process [3260]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
